# Replicating Jersey Recognition results from [General Framework paper](https://github.com/mkoshkina/jersey-number-pipeline)

This notebook was created in Google Colab, so it might be best to run it in Colab. The same steps apply if you want to run it locally, but you need to have Git and Conda installed, and make sure they are in your system Path.

## Install conda

In [ ]:
%pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda --version

conda 24.11.2


## Clone repo and setup their models

In [ ]:
!git clone https://github.com/mkoshkina/jersey-number-pipeline.git

In [ ]:
# setup.py requires this to download model weights from Google Drive links
%pip install gdown

In [ ]:
%cd jersey-number-pipeline

/content/jersey-number-pipeline


In [ ]:
# This removes line 28 which causes an error (reason: running "conda env list" no longer prints '*' to the console, so trying to remove it causes an error)
!sed -i '28d' setup.py

In [ ]:
!python3 setup.py SoccerNet

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 24.11.3

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/vitpose

  added / updated specs:
    - python=3.8


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h4bc722e_7 
  ca-certificates    conda-forge/linux-64::ca-certificates-2024.12.14-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.43-h712a8e2_2 
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 
  libgcc             conda-forge/linux-64::libgcc-14.2.0-h77fa898_1 
  libgcc-ng          conda-forge/linux-64::libgcc-ng-14.2.0-h69a

## Installing the dataset

In [1]:
%pip install SoccerNet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00


Takes around 5-7 minutes to download on Colab, could vary if running locally depending on internet speed. Note that this will need to run every time you restart the Colab instance, so try to add the downloaded files to your Google Drive instead.

In [18]:
from SoccerNet.Downloader import SoccerNetDownloader as SNdl
mySNdl = SNdl(LocalDirectory="")    # change the download location according to the instructions in main repo, when the above code works
mySNdl.downloadDataTask(task="jersey-2023", split=["train","test"])   # or split=["train","test","challenge"] if you also want to download challenge set (this set has no labels)

Unzipping each folder takes about 1 min each

In [21]:
!unzip -q jersey-2023/test.zip -d jersey-2023/

In [22]:
# This prints the number of folders (each folder contains tracklet of one player)
!ls jersey-2023/test/images/ | wc -l

1211


In [23]:
!unzip -q jersey-2023/train.zip -d jersey-2023/

In [24]:
!ls jersey-2023/train/images/ | wc -l

1427


## Format of labels

Labels are listed in a dictionary format:
`{"tracklet-number": jersey-number}`.
If the tracklet does not have any recognizable digits, the jersey number output should be -1.

In [26]:
!head -c 25 jersey-2023/test/test_gt.json

{"0": 4, "1": 93, "2": 25